In [ ]:
import pandas as pd

Starting from the top.

The first-results-sarc-ratio.csv is the basis of the input to the models.

First we check if this corresponds to the input to the models. 

In [ ]:
first_sarc = pd.read_csv('results/first-results-sarc-ratio.csv')
inp_r1 = pd.read_csv('results/input-r2-claim-count.csv')

**We can see in the table, that we do not have any values for when qwen-2.5-1.5b is the model sender and mistral-0.3-7b is the model receiver, when the aggregated label of the receiver model started as literal**

In [ ]:
results = pd.read_csv('results/results.csv')

In [ ]:
results['abs_delta'] = results['delta_sarc'].abs()

Now let's look at the results, where the model receiver started on literal. 

In [ ]:
res = results[results['round1_sarc_ratio'] < 0.5]

In [ ]:
res.groupby(['model_sender', 'model_receiver'])['id'].count()

model_sender    model_receiver
llama-3.2-3b    mistral-0.2-7b     9687
                mistral-0.3-7b       83
                qwen-2.5-1.5b     75796
                qwen-2.5-7b        3707
mistral-0.2-7b  llama-3.2-3b        758
                mistral-0.3-7b        5
                qwen-2.5-1.5b     69598
                qwen-2.5-7b        2036
mistral-0.3-7b  llama-3.2-3b       2365
                mistral-0.2-7b    13664
                qwen-2.5-1.5b     83281
                qwen-2.5-7b        5431
qwen-2.5-1.5b   mistral-0.2-7b      127
                qwen-2.5-7b          80
qwen-2.5-7b     llama-3.2-3b       1586
                mistral-0.2-7b    10005
                mistral-0.3-7b       97
                qwen-2.5-1.5b     77398
Name: id, dtype: int64

It seems like qween-2.5-1.5b is only acting like a sender, towards mistral-0.2-7b and qwen-2.5-7b, when it the receiver had literal as label, and therefore qwen had sarcastic as starting label. 

Let's look into the input of mistral-0.3-7b and see if we have cases where the sarc_ratio_sender is >= 0.75 and the sender model is qwen-2.5-1.5b

In [ ]:
# inp_mistral = pd.read_csv('input_mistral-0.3-7b.csv')
# inp_mistral.columns
# inp_mistral_qwen = inp_mistral[(inp_mistral['model_sender'] == 'qwen-2.5-1.5b')]

In [ ]:
# inp_mistral_qwen.head()

,model_receiver,id,valid_json_count_receiver,sarc_ratio_receiver,model_sender,valid_json_count_sender,sarc_ratio_sender,label_sender,explanation_sender,claim
79767,mistral-0.3-7b,1,10,1.0,qwen-2.5-1.5b,10,0.0,literal,The statement is a straightforward description...,At first I thought he jammed it by inserting h...
79768,mistral-0.3-7b,1,10,1.0,qwen-2.5-1.5b,10,0.0,literal,The statement is straightforward and does not ...,At first I thought he jammed it by inserting h...
79769,mistral-0.3-7b,1,10,1.0,qwen-2.5-1.5b,10,0.0,literal,The statement is not meant to be sarcastic; it...,At first I thought he jammed it by inserting h...
79770,mistral-0.3-7b,1,10,1.0,qwen-2.5-1.5b,10,0.0,literal,The statement is a straightforward description...,At first I thought he jammed it by inserting h...
79771,mistral-0.3-7b,1,10,1.0,qwen-2.5-1.5b,10,0.0,literal,The statement describes an action in a straigh...,At first I thought he jammed it by inserting h...


In [ ]:
# inp_mistral_qwen[inp_mistral_qwen['sarc_ratio_sender']> 0.5]

,model_receiver,id,valid_json_count_receiver,sarc_ratio_receiver,model_sender,valid_json_count_sender,sarc_ratio_sender,label_sender,explanation_sender,claim


Doesn't seem like there is. Let's check if this corresponds to what we see in the two sarc-ratio dfs. 

In [ ]:
first_sarc = pd.read_csv('results/first-results-sarc-ratio.csv')

filtering out invalids

In [ ]:
first_sarc_valid = first_sarc[first_sarc['valid_json_count'] >= 5]
first_sarc_valid

,model,id,valid_json_count,sarc_ratio
0,llama-3.2-1b,1,10,1.0
1,llama-3.2-1b,2,10,1.0
2,llama-3.2-1b,3,10,1.0
3,llama-3.2-1b,6,10,1.0
4,llama-3.2-1b,9,10,1.0
...,...,...,...,...
824719,qwen-2.5-7b,205659,10,1.0
824720,qwen-2.5-7b,205660,10,1.0
824721,qwen-2.5-7b,205661,10,1.0
824722,qwen-2.5-7b,205662,10,1.0


Since this is the first sarc results df, we are interested in the receiver. So we filter for mistral, and for the sarc ratios where mistral has >= 0.75 in sarc ratio.

In [ ]:
first_sarc_valid_mistral = first_sarc_valid[(first_sarc_valid['model'] == 'mistral-0.3-7b') & (first_sarc_valid[first_sarc_valid['sarc_ratio'] >= 0.75])]

In [ ]:
mistral_literal_ids = first_sarc_valid[(first_sarc_valid['model'] == 'mistral-0.3-7b') & (first_sarc_valid['sarc_ratio'] < 0.5)]['id']
mistral_literal_ids

412369        13
412372        18
412388        38
412410        72
412426        90
           ...  
548829    204169
548899    204266
549235    204785
549283    204856
549728    205516
Name: id, Length: 501, dtype: int64

In [ ]:
# Merge on id, but only where model differs
merged = first_sarc_valid.merge(
    first_sarc_valid,
    on="id",
    suffixes=("_a", "_b"),
    how="left"
)

# Filter model mismatch
merged = merged[merged["model_a"] != merged["model_b"]]

# Apply sarc_ratio conditions
filtered = merged[
    ((merged["sarc_ratio_a"] >= 0.75) & (merged["sarc_ratio_b"] <= 0.25)) |
    ((merged["sarc_ratio_a"] <= 0.25) & (merged["sarc_ratio_b"] >= 0.75))
]

In [ ]:
qwen_mistral = filtered[(filtered['model_a'] == 'qwen-2.5-1.5b') & (filtered['model_b'] == 'mistral-0.3-7b')]
qwen_mistral

,model_a,id,valid_json_count_a,sarc_ratio_a,model_b,valid_json_count_b,sarc_ratio_b
3297053,qwen-2.5-1.5b,1,10,0.0,mistral-0.3-7b,10,1.0
3297059,qwen-2.5-1.5b,2,10,0.2,mistral-0.3-7b,10,1.0
3297065,qwen-2.5-1.5b,3,10,0.0,mistral-0.3-7b,10,1.0
3297071,qwen-2.5-1.5b,6,10,0.0,mistral-0.3-7b,10,1.0
3297101,qwen-2.5-1.5b,14,10,0.0,mistral-0.3-7b,10,1.0
...,...,...,...,...,...,...,...
4121532,qwen-2.5-1.5b,205658,10,0.0,mistral-0.3-7b,10,1.0
4121538,qwen-2.5-1.5b,205659,10,0.1,mistral-0.3-7b,10,1.0
4121544,qwen-2.5-1.5b,205660,10,0.0,mistral-0.3-7b,10,1.0
4121550,qwen-2.5-1.5b,205661,10,0.0,mistral-0.3-7b,10,1.0


In [ ]:
inp_r1[(inp_r1['model_sender'] == 'qwen-2.5-1.5b') & (inp_r1['model_receiver'] == 'mistral-0.3-7b')]

,model_sender,model_receiver,label_sender,n_unique_claims
28,qwen-2.5-1.5b,mistral-0.3-7b,literal,83281
29,qwen-2.5-1.5b,mistral-0.3-7b,sarcastic,11134


In [ ]:
inp_mistral = pd.read_csv('input_mistral-0.3-7b.csv')
inp_mistral.columns
inp_mistral_qwen = inp_mistral[(inp_mistral['model_sender'] == 'qwen-2.5-1.5b')]
inp_mistral_qwen.groupby('label_sender')['id'].nunique()

label_sender
literal      83281
sarcastic    11134
Name: id, dtype: int64

In [ ]:
inp_mistral_qwen[inp_mistral_qwen['id'] == 2]

,model_receiver,id,valid_json_count_receiver,sarc_ratio_receiver,model_sender,valid_json_count_sender,sarc_ratio_sender,label_sender,explanation_sender,claim
79777,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,The statement is a straightforward observation...,The Browns don't know shit when it comes to QB's.
79778,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,The statement is a straightforward observation...,The Browns don't know shit when it comes to QB's.
79779,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,The statement is a straightforward observation...,The Browns don't know shit when it comes to QB's.
79780,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,sarcastic,The phrase 'don't know shit' is often used as ...,The Browns don't know shit when it comes to QB's.
79781,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,The statement is a direct observation and does...,The Browns don't know shit when it comes to QB's.
79782,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,sarcastic,The statement suggests that the Browns have a ...,The Browns don't know shit when it comes to QB's.
79783,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,The statement is a straightforward observation...,The Browns don't know shit when it comes to QB's.
79784,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,This statement is not sarcastic because the Br...,The Browns don't know shit when it comes to QB's.
79785,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,The statement is not inherently sarcastic; the...,The Browns don't know shit when it comes to QB's.
79786,mistral-0.3-7b,2,10,1.0,qwen-2.5-1.5b,10,0.2,literal,The statement is a straightforward observation...,The Browns don't know shit when it comes to QB's.


In [ ]:
qwen_sarc_ids = first_sarc_valid[(first_sarc_valid['model'] == 'qwen-2.5-1.5b')]

In [ ]:
qwen_sarc_ids[qwen_sarc_ids['id'] == 2]

,model,id,valid_json_count,sarc_ratio
549817,qwen-2.5-1.5b,2,10,0.2


In [ ]:
first_sarc_valid[(first_sarc_valid['model'] == 'mistral-0.3-7b')]

,model,id,valid_json_count,sarc_ratio
412362,mistral-0.3-7b,1,10,1.0
412363,mistral-0.3-7b,2,10,1.0
412364,mistral-0.3-7b,3,10,1.0
412365,mistral-0.3-7b,6,10,1.0
412366,mistral-0.3-7b,9,10,1.0
...,...,...,...,...
549811,mistral-0.3-7b,205659,10,1.0
549812,mistral-0.3-7b,205660,10,1.0
549813,mistral-0.3-7b,205661,10,1.0
549814,mistral-0.3-7b,205662,10,1.0


In [ ]:
second = pd.read_csv('results/second-results-sarc-ratio.csv')

In [ ]:
second_qwen_mistral = second[(second['model_sender'] == 'qwen-2.5-1.5b') & (second['model_receiver'] == 'mistral-0.3-7b')]

In [ ]:
second_qwen_mistral[second_qwen_mistral['sarc_ratio'] < ]

,model_sender,model_receiver,id,valid_json_count,sarc_ratio
450300,qwen-2.5-1.5b,mistral-0.3-7b,1,10,1.0
450301,qwen-2.5-1.5b,mistral-0.3-7b,2,10,1.0
450302,qwen-2.5-1.5b,mistral-0.3-7b,3,10,0.7
450303,qwen-2.5-1.5b,mistral-0.3-7b,6,10,1.0
450304,qwen-2.5-1.5b,mistral-0.3-7b,14,10,0.4
...,...,...,...,...,...
533576,qwen-2.5-1.5b,mistral-0.3-7b,205658,10,0.9
533577,qwen-2.5-1.5b,mistral-0.3-7b,205659,10,0.9
533578,qwen-2.5-1.5b,mistral-0.3-7b,205660,10,1.0
533579,qwen-2.5-1.5b,mistral-0.3-7b,205661,10,1.0


In [ ]:
results = pd.read_csv('results/results.csv')

In [ ]:
res_qwen_mistral = results[(results['model_sender'] == 'qwen-2.5-1.5b') & (results['model_receiver'] == 'mistral-0.3-7b')]

In [ ]:
results[(results['round1_sarc_ratio'] >= 0.5) & (results['model_sender'] == 'qwen-2.5-1.5b')]['model_receiver'].unique()

array(['llama-3.2-3b', 'qwen-2.5-7b', 'mistral-0.3-7b', 'mistral-0.2-7b'],
      dtype=object)